In [1]:
import sys
import os
import dtale
import dtale.app as dtale_app
dtale_app.JUPYTER_SERVER_PROXY = True
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config
config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}
config = Config(config=config_info)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

A configuration file has been created for you in 
/home/canalyst_candas/canalyst/keys.json. For Excel file downloads and scenario 
mapping, you'll need to add an API key to the 'canalyst_api_key' 
value. Visit https://app.canalyst.com/settings/api-tokens to create 
and retrieve your API key. You will also need to retrieve your 
S3 Access ID and Secret Key and fill in 'canalyst_s3_id' and 
'canalyst_s3_key' with those values, respectively. If using a 
Jupyter Notebook, stop and restart the notebook for the changes to 
take effect. If using a Python/iPython session, quit the current 
session and start a new one.


## JPM Read Through for consumer credit costs to DFS and COF

### First we search for common time series among JPM V and MA with 'credit card sales volume debit payments US'

In [2]:
canalyst_search = cd.Search(config)
df = canalyst_search.search_time_series(ticker=['JPM US','COF US','DFS US'],
                                        unit_type='currency',
                     time_series_description='credit card consumer credit charge-off recoveries')[['ticker','time_series_description','time_series_name','search_matches']]
df= df[df['search_matches']>1]

Building search index...
No.of.unique tickers:  3
No.of.unique time series:  131


In [3]:
df
#these are our three series
#JPM US z_3FRM7X0148_MO_OS_netchargeoffsrecoveries
#COF US z_YN5MQC0170_MO_OS_CreditCardNetchargeoffs	
#DF US z_YDE7B00120_MO_OS_netchargeoffs


,ticker,time_series_description,time_series_name,search_matches
502605,COF US,"Credit Card - Domestic credit card, mm",z_YN5MQC0170_MO_OS_CreditCardDomesticcreditcard,4
502539,COF US,"Credit Card - Provision for credit losses, mm",z_YN5MQC0170_MO_OS_CreditCardProvisionforcreditlosses,3
502547,COF US,"Credit Card - Net charge-offs, mm",z_YN5MQC0170_MO_OS_CreditCardNetchargeoffs,3
502606,COF US,"Credit Card - International card businesses, mm",z_YN5MQC0170_MO_OS_CreditCardInternationalcardbusinesses,3
50232,JPM US,"Net Charge-offs/(recoveries), mm",z_3FRM7X0148_MO_OS_netchargeoffsrecoveries,2
50289,JPM US,"Debit and credit card sales volumes, bn",z_3FRM7X0148_MO_OS_Debitandcreditcardsalesvolumes,2
50223,JPM US,"Cumulative Net Charge-offs/(recoveries), mm",z_3FRM7X0148_MO_OS_cumulativenetchargeoffsrecoveries,2
50448,JPM US,Credit card securitization adjustment,z_3FRM7X0148_MO_OS_Creditcardsecuritizationadjustment,2
50287,JPM US,"Credit card sales volume, bn",z_3FRM7X0148_MO_OS_Creditcardsalesvolume,2
50267,JPM US,"Consumer & Community Banking Provision for credit losses, mm",z_3FRM7X0148_MO_OS_ConsumerCommunityBankingProvisionforcreditlosses,2


### Now that we have our series, let's pull the data from Canalyst

In [4]:
ticker = "JPM US"

In [5]:
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)

In [6]:
df_jpm_credit = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      is_historical=True,
                      time_series_name='z_3FRM7X0148_MO_OS_netchargeoffsrecoveries')[['ticker','time_series_name','period_end_date','value']] 

### Next we get COF data from Canalyst

In [7]:
ticker = "COF US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_cof_credit = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_YN5MQC0170_MO_OS_CreditCardNetchargeoffs')[['ticker','time_series_name','period_end_date','value']]

### Same data pull for DFS

In [12]:
ticker = "DFS US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_dfs_credit = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_YDE7B00120_MO_OS_netchargeoffs')[['ticker','time_series_name','period_end_date','value']]

### Now we use the regress_dataframe function from Candas DataReader

#### JPM credit vs COF credit

In [9]:
df = pd.merge(df_jpm_credit,df_cof_credit,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

Dep. Variable:,value_y,R-squared:,0.710
Model:,OLS,Adj. R-squared:,0.696
Method:,Least Squares,F-statistic:,53.78
Date:,"Mon, 10 Jan 2022",Prob (F-statistic):,2.42e-07
Time:,14:56:05,Log-Likelihood:,-486.41
No. Observations:,24,AIC:,976.8
Df Residuals:,22,BIC:,979.2
Df Model:,1,,
Covariance Type:,nonrobust,,


#### JPM credit vs DFS credit 

In [13]:
df = pd.merge(df_jpm_credit,df_dfs_credit,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

Dep. Variable:,value_y,R-squared:,0.383
Model:,OLS,Adj. R-squared:,0.355
Method:,Least Squares,F-statistic:,13.65
Date:,"Mon, 10 Jan 2022",Prob (F-statistic):,0.00127
Time:,14:56:37,Log-Likelihood:,-478.51
No. Observations:,24,AIC:,961.0
Df Residuals:,22,BIC:,963.4
Df Model:,1,,
Covariance Type:,nonrobust,,
